In [1]:
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset,num2date
import numpy as np
import netCDF4 as ncf
import matplotlib.pyplot as plt
import chardet #处理读取数据乱码的包



ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [2]:
plt.rcParams['font.sans-serif']= ['simhei']
plt.rcParams['axes.unicode_minus']=False
data = ncf.Dataset('F:\HadISST_sst.nc',dtype = np.float32)
print(data['latitude'][30:100])
print(data['longitude'][60:240])
t = data['time']
time=t[:]
timed=num2date(time,t.units)
ms  = str(num2date(time[983],  units=t.units, calendar=t.calendar))[0:10]
print(ms)
#提取1961年到2010年12月的海温
sst = data['sst'][np.arange(983,1692,12),30:100,60:240]
sst[sst<-999]=np.nan
print(sst.shape)
resst = sst[:,range(69,-1,-1),:].reshape(60,12600)


f =  open('F:\ddi','rb')
f_read=f.read()
#转换正常数据
f_charInfo=chardet.detect(f_read)
print(f_charInfo['encoding']) #获取该文件的编码方式，本数据为ascii
f_read_decode=f_read.decode('ascii')
list =f_read_decode.split("\n")
del list[-1]
ff =np.array(list)
print(ff.shape)
#存取三类雨带的年份
A = []
B = []
C = []
for i in range(60):
      if ff[i].strip()=='1. 0. 0.':
        A.append(i)
      if ff[i].strip() == '0. 1. 0.':
          B.append(i)
      if ff[i].strip()== '0. 0. 1.':
          C.append(i)
AA = np.array(A)
BB = np.array(B)
CC = np.array(C)
#根据三类雨带对50年数据分类：
sst1 = resst[AA,:]
sst2 = resst[BB,:]
sst3 = resst[CC,:]
print(sst1.shape,sst2.shape,sst3.shape)
#海温指数
ave1 = np.nanmean(sst1,axis=1)
ave2 = np.nanmean(sst2,axis=1)
ave3 = np.nanmean(sst3,axis=1)

#求距平
ano1 = ave1 - np.nanmean(ave1)
ano2 = ave2 - np.nanmean(ave2)
ano3 = ave3 - np.nanmean(ave3)

plt.figure()
plt.plot(AA+1951,ano1)
plt.title('1类雨型海温距平合成图')
plt.savefig('F:\1.png')
plt.figure()

plt.plot(BB+1951,ano2)
plt.title('2类雨型海温距平合成图')
plt.savefig'F:\2.png')
plt.figure()
plt.plot(CC+1951,ano3)
plt.title('3类雨型海温距平合成图')
plt.savefig('F:\3.png')

NameError: name 'plt' is not defined

In [ ]:
#第二问
#对三类海温求平均
ave11 = np.mean(sst1,axis=0)
ave22 = np.mean(sst2,axis=0)
ave33 = np.mean(sst3,axis=0)
#求距平
ano11 = sst1 - ave11
ano22 = sst2 - ave22
ano33 = sst3 - ave33
#对三类海温求方差
fc11 = np.mean(np.power(ano11,2),axis=0)
fc22 = np.mean(np.power(ano22,2),axis=0)
fc33 = np.mean(np.power(ano33,2),axis=0)
print(fc11.shape)
#求差值
_12 = ave11 - ave22
_13 = ave11 - ave33
_23 = ave22 - ave33
lon = np.linspace(-60,120,180)
lat = np.linspace(-10,60,70)
#画图
class mm:
    def __init__(self):
     
        self.m=Basemap(llcrnrlon=-60,llcrnrlat=-10,
                            urcrnrlat=60,urcrnrlon=120)
        self.m.drawcoastlines()
        self.m.drawcountries()
        self.m.drawparallels(np.linspace(-10,60,5),labels=[True,False,True,False])  
        self.m.drawmeridians(np.linspace(-60,120,7),labels=[False,True,False,True])
        self.X,self.Y = np.meshgrid(lon,lat)
    def fit(self,data):
        
        c = self.m.contourf(self.X,self.Y,data,cmap = plt.cm.RdYlBu)
        self.m.colorbar(c,'right',format = '%.1f')
plt.figure()
mm().fit(_12.reshape(70,180))
plt.title('1类与2类合成差值图')
plt.savefig('F:\4.png')
plt.figure()
mm().fit(_13.reshape(70,180))
plt.title('1类与3类合成差值图')
plt.savefig('F:\5.png')
plt.figure()
mm().fit(_23.reshape(70,180))
plt.title('2类与3类合成差值图')
plt.savefig('F:\6.png')
plt.show()
#此代码第一问并未涉及合成方法，求的是海温指数，得到的是分开的一维的距平图
#合成方法：用1类雨型的年平均值减去所有雨型年份平均值，即距平合成图，算出的数据是二维数据